In [ ]:
import sys

sys.path.append("./optimisation_service")
sys.path.append("./epoch_simulator_lib/build/epoch_py/")

sys.path[0], sys.path[-1] = sys.path[-1], sys.path[0]

In [ ]:
from app.models.epoch_types.site_range_type import Building, Config, GasHeater, GasTypeEnum, Grid, SolarPanel, SiteRange

building = Building(
    COMPONENT_IS_MANDATORY=True,
    scalar_heat_load=[1],
    scalar_electrical_load=[1],
    fabric_intervention_index=[0],
    incumbent=False,
    age=0,
)
config = Config(capex_limit=99999999999, use_boiler_upgrade_scheme=False, general_grant_funding=0.0)
grid = Grid(
    COMPONENT_IS_MANDATORY=True,
    import_headroom=[0.4],
    grid_export=[0],
    grid_import=[140],
    tariff_index=[0, 1, 2, 3],
    export_tariff=[0.2],
    incumbent=True,
    age=0,
)
gas_heater = GasHeater(
    COMPONENT_IS_MANDATORY=False,
    maximum_output=[999999],
    gas_type=[GasTypeEnum.NATURAL_GAS],
    boiler_efficiency=[0.95],
    incumbent=True,
    age=0,
)
solar_panels = [
    SolarPanel(
        COMPONENT_IS_MANDATORY=False, yield_scalar=[10, 20, 30, 40, 50], yield_index=[0], incumbent=False, age=0, lifetime=25
    )
]

site_range = SiteRange(building=building, config=config, grid=grid, gas_heater=gas_heater, solar_panels=solar_panels)

In [ ]:
import datetime
from pathlib import Path
from app.models.core import Site
from app.models.site_data import LegacySiteMetaData
from tests.conftest import load_epoch_data_from_file
from app.models.epoch_types.config import Config
from app.internal.constraints import apply_default_constraints
from app.models.constraints import Bounds
from app.models.metrics import Metric

site_ids = ["amcott_house", "bircotes_leisure_centre"]
portfolio = []
for site_id in site_ids:
    start_ts = datetime.datetime(year=2022, month=1, day=1, hour=0, tzinfo=datetime.UTC)
    end_ts = datetime.datetime(year=2023, month=1, day=1, hour=0, tzinfo=datetime.UTC)
    site_data = LegacySiteMetaData(site_id=site_id, start_ts=start_ts, end_ts=end_ts)

    site = Site(name=site_id, site_range=site_range, site_data=site_data, constraints={}, config=Config())

    epoch_data = load_epoch_data_from_file(path=Path("optimisation_service", "tests", "data", site_id, "epoch_data.json"))

    site._epoch_data = epoch_data

    portfolio.append(site)

constraints = {Metric.capex: Bounds(max=100000)}

portfolio, constraints = apply_default_constraints(portfolio, constraints)

objectives = [Metric.npv_balance, Metric.carbon_balance_total]

In [ ]:
from app.internal.NSGA2 import NSGA2

In [ ]:
nsga = NSGA2(pop_size=512, n_offsprings=256, period=5)

nsga_res = nsga.run(objectives=objectives, constraints=constraints, portfolio=portfolio)

n_gen  |  n_eval  | n_nds  |     cv_min    |     cv_avg    |      eps      |   indicator  
     1 |      389 |      2 |  0.000000E+00 |  7.524261E+04 |             - |             -
     2 |      645 |      3 |  0.000000E+00 |  4.690583E+04 |  0.0815275615 |         ideal
     3 |      901 |      3 |  0.000000E+00 |  2.619396E+04 |  9.1480943870 |         nadir
     4 |     1157 |      2 |  0.000000E+00 |  7.625330E+03 |  2.2216001272 |         nadir
     5 |     1411 |      3 |  0.000000E+00 |  0.000000E+00 |  0.2286269023 |             f
     6 |     1663 |      3 |  0.000000E+00 |  0.000000E+00 |  0.000000E+00 |             f
     7 |     1918 |      3 |  0.000000E+00 |  0.000000E+00 |  0.000000E+00 |             f
     8 |     2174 |      3 |  0.000000E+00 |  0.000000E+00 |  0.000000E+00 |             f
     9 |     2427 |      3 |  0.000000E+00 |  0.000000E+00 |  0.000000E+00 |             f
    10 |     2682 |      3 |  0.000000E+00 |  0.000000E+00 |  0.000000E+00 |             f

In [ ]:
nsga_res.solutions[2].scenario["amcott_house"].scenario

AnnotatedTaskData(building=Building(scalar_heat_load=1.0, scalar_electrical_load=1.0, fabric_intervention_index=0, floor_area=None, incumbent=False, age=0.0, lifetime=30.0), data_centre=None, domestic_hot_water=None, electric_vehicles=None, energy_storage_system=None, gas_heater=GasHeater(maximum_output=999999.0, gas_type=<GasType.NATURAL_GAS: 'NATURAL_GAS'>, boiler_efficiency=0.95, fixed_gas_price=0.068, incumbent=True, age=0.0, lifetime=10.0), grid=Grid(grid_export=0.0, grid_import=140.0, import_headroom=0.4, tariff_index=1, export_tariff=0.2, incumbent=True, age=0.0, lifetime=25.0), heat_pump=None, mop=None, solar_panels=[RepeatAnnotatedSolarPanel(yield_scalar=20.0, yield_index=0, incumbent=False, age=0.0, lifetime=25.0, index_tracker=0)])

In [ ]:
from app.internal.bayesian.simplified_bo import Bayesian

/home/will/git_repos/EPOCH/.venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
bo = Bayesian()

res = bo.run(objectives=objectives, constraints=constraints, portfolio=portfolio)

[36237.37622241069, 63762.62377758931] [[0.1586617320612697, 0.8413382679387302], [0.26397982423277133, 0.7360201757672287]]
n_gen  |  n_eval  | n_nds  |     cv_min    |     cv_avg    |      eps      |   indicator  
     1 |       45 |      1 |  0.000000E+00 |  4.567565E+04 |             - |             -
     2 |       48 |      1 |  0.000000E+00 |  4.523978E+04 |  0.000000E+00 |             f
     3 |       48 |      1 |  0.000000E+00 |  4.523978E+04 |  0.000000E+00 |             f
n_gen  |  n_eval  | n_nds  |     cv_min    |     cv_avg    |      eps      |   indicator  
     1 |       46 |      1 |  0.000000E+00 |  3.413816E+04 |             - |             -
     2 |       48 |      1 |  0.000000E+00 |  3.413816E+04 |  0.000000E+00 |             f
     3 |       48 |      1 |  0.000000E+00 |  3.413816E+04 |  0.000000E+00 |             f
[16917.486234860553, 83082.51376513945] [[0.5814541407925545, 0.41854585920744547], [0.5424921214776075, 0.4575078785223925]]
n_gen  |  n_eval  | n

RuntimeError: max(): Expected reduction dim to be specified for input.numel() == 0. Specify the reduction dim with the 'dim' argument.

In [ ]:
len(res.solutions)

1

In [ ]:
res.solutions

[PortfolioSolution(scenario={'amcott_house': SiteSolution(scenario=AnnotatedTaskData(building=Building(scalar_heat_load=1.0, scalar_electrical_load=1.0, fabric_intervention_index=0, floor_area=None, incumbent=False, age=0.0, lifetime=30.0), data_centre=None, domestic_hot_water=None, electric_vehicles=None, energy_storage_system=None, gas_heater=GasHeater(maximum_output=999999.0, gas_type=<GasType.NATURAL_GAS: 'NATURAL_GAS'>, boiler_efficiency=0.95, fixed_gas_price=0.068, incumbent=True, age=0.0, lifetime=10.0), grid=Grid(grid_export=0.0, grid_import=140.0, import_headroom=0.4, tariff_index=1, export_tariff=0.2, incumbent=True, age=0.0, lifetime=25.0), heat_pump=None, mop=None, solar_panels=[RepeatAnnotatedSolarPanel(yield_scalar=10.0, yield_index=0, incumbent=False, age=0.0, lifetime=25.0, index_tracker=0)]), metric_values={<Metric.meter_balance: 'meter_balance'>: 3825.455078125, <Metric.operating_balance: 'operating_balance'>: 3805.455078125, <Metric.cost_balance: 'cost_balance'>: 318